<a href="https://colab.research.google.com/github/cedro3/average_face/blob/main/AverageFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#テーマ：平均顔
・扱える写真の種類は、**jpg**のみです。１人の顔が写っている写真を使用して下さい\
・画像のアップロード→顔画像の切り出し→ベクトル逆算→平均顔生成、という手順で行います



### 1.自分のPCをマシンに接続
**右上にある接続ボタンをクリック　→　接続完了で緑色のチェックマーク表示**

In [ ]:
#@title 2.マシンにソフトをインストール（数分程度かかります）
import os
os.chdir('/content')
CODE_DIR = 'encoder4editing'

!git clone https://github.com/cedro3/average_face.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
os.chdir(f'./{CODE_DIR}')

from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp  # we use the pSp framework to load the e4e encoder.

%load_ext autoreload
%autoreload 2

# 学習済みパラメータのダウンロード
import os
import gdown
os.makedirs('pretrained_models', exist_ok=True)
gdown.download('https://drive.google.com/u/1/uc?id=1Du_8FzOPKJhk6aJmiOBhAWVe3_6vAyET', 'pretrained_models/e4e_ffhq_encode.pt', quiet=False)

# ランドマークデータのダウンロード
! wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
! bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

# モデルに学習済みパラメータをロード
model_path = 'pretrained_models/e4e_ffhq_encode.pt'  ####
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')


# --- ライブラリーインポート＆関数定義 ---
%tensorflow_version 1.x
import numpy as np
import scipy.ndimage
import os
import PIL.Image
import sys
import bz2
from keras.utils import get_file
import dlib
import argparse
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import re
import projector
import pretrained_networks
from training import dataset
from training import misc
import matplotlib.pyplot as plt
from tqdm import trange


# -------------- フォルダー内画像表示 ---------------
def display_pic(folder):
    fig = plt.figure(figsize=(40, 40))
    files = os.listdir(folder)
    files.sort()
    for i, file in enumerate(files):
        img = Image.open(folder+'/'+file)    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        fig.tight_layout()
        ax.set_xlabel(str(i+1), fontsize=30)               
    plt.show()
    plt.close()  

# -------------- ベクトルから画像を生成・保存 -------------
def vec2pic(vec_syn, dir):
  
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'  
    
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = True 
    Gs_syn_kwargs.truncation_psi = 0.5

    for i in range(len(vec_syn)):
        vec = vec_syn[i].reshape(1,18,512)
        image =  Gs.components.synthesis.run(vec, **Gs_syn_kwargs)        
        img = PIL.Image.fromarray(image[0])
        img.save(dir+str(i+1).zfill(3)+'.jpg') 

### 3.左のウインドウを開く
**左端のファイルシンボルをクリックし、encoder4editingフォルダーの先頭をクリックして開き、さらにその中のimagesフォルダーの先頭をクリックして開く**\
＊左のウインドウは実際と表示にタイムラグがあるので、必要に応じて左上にある更新ボタンを押す

**------------- プログラム本体 --------------**

### 4.フォルダーを作成する
**imagesフォルダーの右端の３つの点をクリックし、「新しいフォルダ」を選択**\
**適当なフォルダー名（例えば'test1'）を入力してリターン**

### 5.自分の用意した顔画像（jpg）をアップロード
**自分の作成したフォルダー（例えば'test1'）の右端の３つの点をクリックし、「アップロード」を選択**\
＊必ず、jpg画像を使って下さい。png画像では上手く行きません\
＊複数の画像を選択すると上手くアップロードできない場合は、まず１枚アップロードすると、その後複数でアップロードできます


In [ ]:
#@title 6.顔画像の切り出し（folderにフォルダー名を指定）

folder = 'test1' #@param {type:"string"}
path = './images/'+folder

import os
import shutil
from tqdm import tqdm

if os.path.isdir('align'):
     shutil.rmtree('align')
os.makedirs('align', exist_ok=True)

def run_alignment(image_path):
  import dlib
  from utils.alignment import align_face
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor) 
  return aligned_image 

files = sorted(os.listdir(path))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = run_alignment(path+'/'+file)
  input_image.resize((256,256))
  input_image.save('./align/'+file)

display_pic('align')

### 7.切り出した顔画像のチェック
**6.で表示された画像が全て狙ったものになっているか確認し、上手く切り出せていない顔画像があれば4.に戻って再度進める**

In [ ]:
#@title 8.ベクトルの逆算
if os.path.isdir('vec'):
     shutil.rmtree('vec')
os.makedirs('vec', exist_ok=True)

img_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

path = './align'
files = sorted(os.listdir(path))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = Image.open(path+'/'+file)
  transformed_image = img_transforms(input_image)
  with torch.no_grad():
     images, latents = net(transformed_image.unsqueeze(0).to('cuda').float(), randomize_noise=False, return_latents=True)
     result_image, latent = images[0], latents[0]
     torch.save(latents, './vec/'+file[:-4]+'.pt') # vec  保存


In [ ]:
#@title 9.平均顔の生成

# フォルダー・リセット
import os
import shutil
if os.path.isdir('vec_avg'):
     shutil.rmtree('vec_avg')
if os.path.isdir('vec_avg_pic'):
     shutil.rmtree('vec_avg_pic')
os.makedirs('vec_avg', exist_ok=True)
os.makedirs('vec_avg_pic', exist_ok=True)

# ベクトルの平均処理
import glob 
files = glob.glob('./vec/*.pt')
files.sort()

avg = 0
for i, file in enumerate(files):
     latent = torch.load(file)
     avg = (i*avg+latent)/(i+1)
     torch.save(avg, './vec_avg/'+str(i+1).zfill(3)+'.pt') # ベクトルを保存
     if i == 0:
        result = avg
     else:
        result = torch.cat((result, avg),0)
vec = result.to('cpu').detach().numpy().copy()

vec_one = vec[-1].reshape(-1,18,512)

# ベクトルから画像を生成し保存
dir = 'vec_avg_pic/'
vec2pic(vec_one, dir)

# 保存した画像を表示
display_pic('vec_avg_pic')

### 10.平均顔のダウンロード
**生成した平均顔は、vec_avg_picに001.jpgで保存されますので、右端の３つの点をクリックし、ダウンロードを選択**\
＊左のウインドウは実際と表示にタイムラグがあるので、必要に応じて左上にある更新ボタンを押す

### 以下、4〜10の繰り返しです

### 【備考】
・このノートを１回実行・終了してから、**すぐ再度実行**する場合は、前の記憶が不完全に残っていて上手く動かないことがあります\
・その場合は、右上にある**「ランタイム／ランタイムを出荷時設定にリセット」**をクリックしてから、最初から操作を行います